In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Tạo các list chứa các url mà ta sẽ lấy dữ liệu


Tạo một list chứa các mùa giải mà ta sẽ lấy dữ liệu

In [59]:
# Đọc nội dung từ tệp season.txt
with open("season.txt", "r") as file:
    content = file.read()

# Tách các dòng thành danh sách các mùa giải
seasons_list = content.split("\n")

# Loại bỏ khoảng trắng và chuyển thành chữ thường
seasons_list = [season.strip().lower() for season in seasons_list if season.strip()]

# In danh sách mùa giải
print(seasons_list)

['lck spring 2020', 'lck summer 2020', 'lpl spring 2020', 'lpl summer 2020', 'lcs spring 2020', 'lcs summer 2020', 'lec spring 2020', 'lec summer 2020', 'vcs spring 2020', 'vcs summer 2020', 'lck spring 2021', 'lck summer 2021', 'lpl spring 2021', 'lpl summer 2021', 'lcs spring 2021', 'lcs summer 2021', 'lec spring 2021', 'lec summer 2021', 'vcs spring 2021', 'vcs summer 2021', 'lck spring 2022', 'lck summer 2022', 'lpl spring 2022', 'lpl summer 2022', 'lcs spring 2022', 'lcs summer 2022', 'lec spring 2022', 'lec summer 2022', 'vcs spring 2022', 'vcs summer 2022', 'lck spring 2023', 'lck summer 2023', 'lpl spring 2023', 'lpl summer 2023', 'lcs spring 2023', 'lcs summer 2023', 'lec spring 2023', 'lec summer 2023', 'vcs spring 2023', 'vcs summer 2023']


Tạo một list chứa các url dẫn đến thông số của mỗi đội trong từng mùa giải


In [37]:
# Khởi tạo mảng urls rỗng
urls = []
tmp_url = "https://gol.gg/teams/list/season-ALL/split-ALL/tournament-"

# Đọc từng dòng trong file season.txt và thêm vào mảng urls
with open('season.txt', 'r') as file:
    for line in file:
        # Loại bỏ ký tự xuống dòng và thay thế khoảng trắng bằng %20
        formatted_line = line.strip().replace(' ', '%20') + '/'

        # Thêm vào mảng urls
        urls.append(tmp_url + formatted_line)

# In mảng urls để kiểm tra
for url in urls:
    print(url)


https://gol.gg/teams/list/season-ALL/split-ALL/tournament-LCK%20spring%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-lCK%20summer%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-LPL%20spring%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-lPL%20summer%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-LCS%20spring%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-lCS%20summer%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-LEC%20spring%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-lEC%20summer%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-VCS%20spring%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-VCS%20summer%202020/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-LCK%20spring%202021/
https://gol.gg/teams/list/season-ALL/split-ALL/tournament-lCK%20summer%202021/
https://gol.gg/teams/list/season-ALL/split-ALL/tourn

Tạo một list chứa các url dẫn đến lịch sử đấu của từng đội trong các mùa giải mà ta sẽ lấy dữ liệu

In [60]:
# thu thập lịch sử đấu của mỗi đội trong các mùa 10, 11, 12, 13 tại các khu vực VCS, LEC, LCS, LPL, LCK
urls2 = []
tmp_url = "https://gol.gg/tournament/tournament-matchlist/"

with open('season.txt', 'r') as file:
    for line in file:
        # Loại bỏ ký tự xuống dòng và thay thế khoảng trắng bằng %20
        formatted_line = line.strip().replace(' ', '%20') + '/'

        # Thêm vào mảng urls
        urls2.append(tmp_url + formatted_line)

# In mảng urls để kiểm tra
for url in urls2:
    print(url)

https://gol.gg/tournament/tournament-matchlist/LCK%20spring%202020/
https://gol.gg/tournament/tournament-matchlist/lCK%20summer%202020/
https://gol.gg/tournament/tournament-matchlist/LPL%20spring%202020/
https://gol.gg/tournament/tournament-matchlist/lPL%20summer%202020/
https://gol.gg/tournament/tournament-matchlist/LCS%20spring%202020/
https://gol.gg/tournament/tournament-matchlist/lCS%20summer%202020/
https://gol.gg/tournament/tournament-matchlist/LEC%20spring%202020/
https://gol.gg/tournament/tournament-matchlist/lEC%20summer%202020/
https://gol.gg/tournament/tournament-matchlist/VCS%20spring%202020/
https://gol.gg/tournament/tournament-matchlist/VCS%20summer%202020/
https://gol.gg/tournament/tournament-matchlist/LCK%20spring%202021/
https://gol.gg/tournament/tournament-matchlist/lCK%20summer%202021/
https://gol.gg/tournament/tournament-matchlist/LPL%20spring%202021/
https://gol.gg/tournament/tournament-matchlist/lPL%20summer%202021/
https://gol.gg/tournament/tournament-matchlist/L

## Thực hiện thu thập dữ liệu

Thu thập thông số của các đội trong các mùa 10, 11, 12, 13 tại các khu vực VCS, LEC, LCS, LPL, LCk

In [52]:
data = []
columns = []
with requests.session() as sr:
    sr.headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:78.0) Gecko/20100101 Firefox/78.0'})
    for i in range(len(urls)):
        partUrl = sr.get(urls[i])
        soup = BeautifulSoup(partUrl.content,'html.parser')

        table = soup.find("table", class_="table_list playerslist tablesaw trhover")
        if len(columns) == 0:
            columns = [j.get_text(strip=True) for j in table.find("thead").find_all("th") if j]
            columns.append("esportSS")
        if table.find("thead"):
            table.find("thead").extract()
        for tr in table.find_all("tr"):
            row_data = [td.get_text(strip=True) for td in tr.find_all("td")]
            if len(row_data) == len(columns) - 1:
                row_data.append(seasons_list[i])
                data.append(row_data)
    
    LOLteams = pd.DataFrame(data, columns=columns)

In [13]:
LOLteams

,Name,Season,Region,Games,Win rate,K:D,GPM,GDM,Game duration,Kills / game,...,TD@15,GD@15,PPG,NASHPG,NASH%,CSM,DPM,WPM,VWPM,WCPM
0,Afreeca Freecs,S10,KR,46,41.3%,0.82,1667,-77,35:00,10.7,...,0.0,-475,-,0.54,42.5,31.0,1730,3.8,1.52,1.89
1,DAMWON Gaming,S10,KR,43,51.2%,1.01,1784,54,33:31,11.5,...,0.0,15,-,0.74,49.5,33.7,1800,3.7,1.44,1.79
2,DragonX,S10,KR,46,67.4%,1.42,1826,167,32:45,13.2,...,0.3,1434,-,0.70,62.4,34.2,1969,3.6,1.47,1.81
3,Gen.G eSports,S10,KR,44,70.5%,1.59,1791,161,33:34,12.3,...,0.2,1203,-,0.80,69.9,33.1,1726,3.8,1.47,1.82
4,Griffin,S10,KR,45,35.6%,0.83,1633,-133,34:03,8.9,...,-0.5,-842,-,0.27,25.7,32.0,1532,3.9,1.60,1.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,Saigon Buffalo,S13,VN,27,55.6%,0.96,1846,-5,29:38,13.1,...,-0.1,39,5.15,0.81,51.9,32.5,2370,3.3,1.29,1.54
418,SBTC Esports,S13,VN,2,0.0%,0.55,1845,-180,31:20,12.0,...,-0.5,-2021,4.00,1.00,75.0,32.9,2239,3.3,1.34,1.74
419,Team Flash,S13,VN,29,41.4%,0.86,1800,-88,32:38,11.7,...,-0.2,-357,4.59,0.76,44.3,32.8,2084,3.7,1.69,1.70
420,Team Secret,S13,VN,26,50.0%,1.00,1848,-21,31:11,13.6,...,0.2,-448,4.62,0.65,41.0,33.4,2374,3.5,1.50,1.79


Mỗi đội sẽ có các dữ liệu sau:
<style>
    table {
        margin: 0 auto;
    }
</style>

| Variable      | Meaning                               |
|---------------|---------------------------------------|
| CSM           | Creep score per minute                |
| DPM           | Damage per minute                     |
| DRA%          | Percentage of dragons killed          |
| DRA@15        | Dragons at 15                         |
| Deaths/game   | Deaths of all player per game         |
| FB%           | First blood percentage                |
| FT%           | First turret percentage               |
| GD@15         | Gold difference @ 15                  |
| GDM           | Gold difference per minute            |
| GPM           | Gold per minute                       |
| Game duration | Game duration                         |
| Games         | Number of games                       |
| HER%          | Percentage of heralds killed          |
| HERPG         | Heralds per game                      |
| K:D           | Average Kills/Deaths per game         |
| Kills/game    | Kills of all player per game          |
| NASH%         | Percentage of Barons killed           |
| NASHPG        | Barons per game                       |
| Name          | Name of the team                      |
| Region        | Region of the team                    |
| Season        | Season                                |
| TD@15         | Tower differential @ 15               |
| Towers killed | Towers killed per game                |
| Towers lost   | Towers lost per game                  |
| VWPM          | Vision wards per minute               |
| WCPM          | Wards cleared per minute              |
| WPM           | Wards per minute                      |
| Win rate      | Win rate of the team                  |    


Thu thập lịch sử đấu của mỗi đội trong các mùa 10, 11, 12, 13 tại các khu vực VCS, LEC, LCS, LPL, LCK

In [44]:
data = []
columns = []
with requests.session() as sr:
    sr.headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:78.0) Gecko/20100101 Firefox/78.0'})
    for i in range(len(urls2)):
        partUrl = sr.get(urls2[i])
        soup = BeautifulSoup(partUrl.content,'html.parser')

        table = soup.find("table", class_="table_list footable toggle-square-filled")
        if len(columns) == 0:
            columns = [j.get_text(strip=True) for j in table.find("thead").find_all("th") if j]
            columns.append('esportSS')
        if table.find("thead"):
            table.find("thead").extract()
        for tr in table.find_all("tr"):
            row_data = [td.get_text(strip=True) for td in tr.find_all("td")]
            if len(row_data) == len(columns) - 1:
                row_data.append(seasons_list[i])
                data.append(row_data)
    LOLHistorys = pd.DataFrame(data, columns=columns)

In [49]:
LOLHistorys

,Game,Team 1,Score,Team 2,Week,Patch,Date,esportSS
0,APK Prince vs DragonX,DragonX,2 - 1,SeolHaeOne Prince,WEEK9,10.7,2020-04-16,lck spring 2020
1,Griffin vs HLE,Griffin,2 - 0,Hanwha Life eSports,WEEK9,10.7,2020-04-16,lck spring 2020
2,DWG vs SB,DAMWON Gaming,2 - 1,Sandbox Gaming,WEEK9,10.7,2020-04-15,lck spring 2020
3,AF vs T1,T1,2 - 0,Afreeca Freecs,WEEK9,10.7,2020-04-15,lck spring 2020
4,GEN vs KT Rolster,Gen.G eSports,2 - 1,KT Rolster,WEEK9,10.7,2020-04-15,lck spring 2020
...,...,...,...,...,...,...,...,...
3549,MBE vs Team Whales,MGN Box Esports,1 - 2,Team Whales,WEEK1,13.12,2023-06-25,vcs summer 2023
3550,Team Secret vs SGB,Team Secret,2 - 0,Saigon Buffalo,WEEK1,13.12,2023-06-24,vcs summer 2023
3551,Team Flash vs CES,Team Flash,2 - 0,Cerberus Esports,WEEK1,13.12,2023-06-24,vcs summer 2023
3552,SGB vs Team Whales,Saigon Buffalo,1 - 2,Team Whales,WEEK1,13.12,2023-06-23,vcs summer 2023


## Trộn các dataframe để thu được file dữ liệu cuối cùng

Đặt lại tên cho các cột trong LOLHistorys

In [47]:
LOLHistorys.columns = ['Game','Team 1','Score','Team 2','Week','Patch','Date','esportSS']

In [53]:
LOLteams.columns

Index(['Name', 'Season', 'Region', 'Games', 'Win rate', 'K:D', 'GPM', 'GDM',
       'Game duration', 'Kills / game', 'Deaths / game', 'Towers killed',
       'Towers lost', 'FB%', 'FT%', 'DRAPG', 'DRA%', 'VGPG', 'HERPG', 'HER%',
       'DRA@15', 'TD@15', 'GD@15', 'PPG', 'NASHPG', 'NASH%', 'CSM', 'DPM',
       'WPM', 'VWPM', 'WCPM', 'esportSS'],
      dtype='object')

Thực hiện trộn các dataframe

In [61]:
LOLHistorys['Patch_short'] = LOLHistorys['Patch'].str.slice(stop=2)
LOLteams['Season_short'] = LOLteams['Season'].str.slice(start=-2)
merged_df = pd.merge(LOLHistorys, LOLteams, left_on=['Team 1', 'Patch_short','esportSS'], right_on=['Name', 'Season_short','esportSS'], how='inner')
merged_df = pd.merge(merged_df, LOLteams, left_on=['Team 2', 'Patch_short','esportSS'], right_on=['Name', 'Season_short','esportSS'], how='inner')

In [62]:
merged_df

,Game,Team 1,Score,Team 2,Week,Patch,Date,esportSS,Patch_short,Name_x,...,GD@15_y,PPG_y,NASHPG_y,NASH%_y,CSM_y,DPM_y,WPM_y,VWPM_y,WCPM_y,Season_short_y
0,APK Prince vs DragonX,DragonX,2 - 1,SeolHaeOne Prince,WEEK9,10.7,2020-04-16,lck spring 2020,10,DragonX,...,-925,-,0.67,50.5,31.2,1827,3.7,1.44,1.78,10
1,Griffin vs APK Prince,Griffin,2 - 1,SeolHaeOne Prince,WEEK2,10.2,2020-02-14,lck spring 2020,10,Griffin,...,-925,-,0.67,50.5,31.2,1827,3.7,1.44,1.78,10
2,DWG vs APK Prince,DAMWON Gaming,2 - 0,SeolHaeOne Prince,WEEK6,10.5,2020-03-28,lck spring 2020,10,DAMWON Gaming,...,-925,-,0.67,50.5,31.2,1827,3.7,1.44,1.78,10
3,T1 vs APK Prince,T1,2 - 0,SeolHaeOne Prince,WEEK6,10.5,2020-03-26,lck spring 2020,10,T1,...,-925,-,0.67,50.5,31.2,1827,3.7,1.44,1.78,10
4,GEN vs APK Prince,Gen.G eSports,2 - 0,SeolHaeOne Prince,WEEK4,10.3,2020-03-01,lck spring 2020,10,Gen.G eSports,...,-925,-,0.67,50.5,31.2,1827,3.7,1.44,1.78,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3548,CES vs GAM Esports,Cerberus Esports,2 - 1,GAM Esports,WEEK7,13.14,2023-08-05,vcs summer 2023,13,Cerberus Esports,...,941,6.07,0.73,62.1,34.3,2383,3.5,1.31,1.46,13
3549,Team Flash vs GAM Esports,Team Flash,2 - 0,GAM Esports,WEEK2,13.12,2023-06-29,vcs summer 2023,13,Team Flash,...,941,6.07,0.73,62.1,34.3,2383,3.5,1.31,1.46,13
3550,SGB vs GAM Esports,Saigon Buffalo,0 - 2,GAM Esports,WEEK7,13.14,2023-08-04,vcs summer 2023,13,Saigon Buffalo,...,941,6.07,0.73,62.1,34.3,2383,3.5,1.31,1.46,13
3551,Team Secret vs GAM Esports,Team Secret,0 - 2,GAM Esports,WEEK4,13.13,2023-07-15,vcs summer 2023,13,Team Secret,...,941,6.07,0.73,62.1,34.3,2383,3.5,1.31,1.46,13


Lưu dataframe vào một file csv để sử dụng

In [58]:
merged_df.to_csv("data.csv", index=False)